#Week 3 Segmenting and Clustering Neighborhoods

##1.	Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe
To create the dataframe:
•	The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
•	Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
•	More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that - M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the - neighborhoods separated with a comma as shown in row 11 in the above table.
•	If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
•	Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
•	In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [1]:
##Load the dependencies
###Install packages, comment out once installed.
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes 
#!conda install -c conda-forge beautifulsoup4 --yes


In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

import folium # map rendering library
from bs4 import BeautifulSoup
print('Libraries imported.')


Libraries imported.


/Users/ggoddard/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
##Get the data from Wikipedia
web_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(web_url,'html.parser')
Data_table = soup.find('table',{'class':'wikitable sortable'})
Data_table
print(Data_table.tr.text) #Check progress so far.



Postal Code

Borough

Neighborhood



In [4]:
##Create the Headers
df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])


In [5]:
##Getting the values from the  table into the dataframe
for tr_cell in Data_table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head() #Check dataframe so far.


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
## Remove the not assigned Boroughs  
df=df[df['Borough']!='Not assigned']
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)




In [7]:
##Sort, reindex and merge
##Sort and combine multiple neighborhoods within a postal code into one row each
tmp_df=df.groupby('PostalCode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
##reset index
tmp_df=tmp_df.reset_index(drop=False)
##Change column name to allow subequent name
tmp_df.rename(columns={'Neighborhood':'Neighborhood_merged'},inplace=True)
## Combine the original data frame which still has Borough with the sorted dataframe to have all three columns again
df_merge = pd.merge(df, tmp_df, on='PostalCode')
df_merge.head()

,PostalCode,Borough,Neighborhood,Neighborhood_merged
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"


In [8]:
## Drop the duplicate column
df_merge.drop(['Neighborhood'],axis=1,inplace=True)
## Drop the duplicate rows
df_merge.drop_duplicates(inplace=True)
## Rename to the Neighborhood column name
df_merge.rename(columns={'Neighborhood_merged':'Neighborhood'},inplace=True)



In [9]:
df_merge.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df_merge.shape

(103, 3)

##PART 2
Continuation of the first part
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.
In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


In [11]:
#!pip install geocoder #using PIP to install geocoder, remark once installed once.
import geocoder #import geocoder

def get_latlong(postal_code): #defining a new function to allow calling it
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):#Loop until we get an answer
        #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code)) #Google didn't work well
        g = geocoder.osm('{}, Toronto, Ontario'.format(postal_code)) #OSM worked much better 
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return postal_code, latitude,longitude



In [12]:
string="M1A"
get_latlong(string) #trying it out. Proved to be super slow when passing all the lines


('M1A', 43.6534817, -79.3839347)

In [14]:
df_latlong= pd.read_csv("http://cocl.us/Geospatial_data") #Load data directly from predownloaded csv
df_latlong.rename(columns={'Postal Code':'PostalCode'}, inplace=True) #Match up the column ID's
df_latlong.set_index("PostalCode")
df_merge.set_index("PostalCode")
df_part2=pd.merge(df_merge, df_latlong) #Combine the Postalcode, Borough, Neighborhood data from the previous DF with the Lat and Long from the other
df_part2.head(12) # Although the sorting is different so the first 12 rows are different, the records contain the same information.

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
